In [15]:
# citation: https://drive.google.com/drive/folders/1idfa8y7esf7usGo7SSxsH4iKBECEPFNr?usp=share_link 
# citation: https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

# from google.colab import drive


In [16]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

# class MultiClassImageTestDataset(Dataset):
#     def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
#         self.super_map_df = super_map_df
#         self.sub_map_df = sub_map_df
#         self.img_dir = img_dir
#         self.transform = transform

#     def __len__(self): # Count files in img_dir
#         return len([fname for fname in os.listdir(self.img_dir)])

#     def __getitem__(self, idx):
#         img_name = str(idx) + '.jpg'
#         img_path = os.path.join(self.img_dir, img_name)
#         image = Image.open(img_path).convert('RGB')


#         super_idx = self.ann_df['superclass_index'][idx]
#         super_label = self.super_map_df['class'][super_idx]

#         sub_idx = self.ann_df['subclass_index'][idx]
#         sub_label = self.sub_map_df['class'][sub_idx]

#         if self.transform:
#             image = self.transform(image)

#         return image, super_idx, super_label, sub_idx, sub_label, img_name

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [17]:
# # load the data from google drive and load into dataframes

# drive.mount('/content/drive')

file_path = 'Released_Data/'
file_path_new_data = 'new_data/'
# sub_class = 'subclass_mapping.csv'
# super_class = 'superclass_mapping.csv'
# train_data = 'train_data.csv'

# # Now you can read or manipulate the file
# df_sub = pd.read_csv(file_path+'subclass_mapping.csv')
# df_subclass = pd.read_csv(file_path+sub_class)
# df_superclass = pd.read_csv(file_path+super_class)
# df_train = pd.read_csv(file_path+train_data)

# # Example: Print the content
# df_subclass.head()

# train_ann_df = pd.read_csv(file_path + 'train_data.csv')
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

# train_img_dir = file_path + 'train_shuffle'
train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

# count_subclass_87 = len(train_ann_df[train_ann_df['subclass_index'] == 87]) by default, there are 0 novel classes in training, so we should create our own

# all classes with subclass 45, 57, and 29, 78 -> should get subclass label of 87 (novel)

# target_classes = [45, 57, 29, 78]

# Update the subclass_index to 87 for the specified classes
# train_ann_df.loc[train_ann_df['subclass_index'].isin(target_classes), 'subclass_index'] = 87


# image_preprocessing = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0), std=(1)),
# ])


In [18]:
# perform data manipulation: rotation, swapping, etc so that we can increase the size of the data set

# Suggestion:
# You should aim to train a generalizable model with all the techniques we have discussed
# so far (e.g., data augmentation, weight decay, etc.) Other tricks may include potentially
# building your local validation set for testing the model's generalization ability.

# Data augmentation and normalization
from torchvision import transforms

# # Training data transformations
# transform_train = transforms.Compose([
#     transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop with scaling
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Testing data transformations
# transform_test = transforms.Compose([
#     transforms.Resize(224),  # Resize to 224x224
#     transforms.CenterCrop(224),  # Center crop to maintain the aspect ratio
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])


# Training data transformations
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(64),  # Random crop with scaling
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Testing data transformations
transform_test = transforms.Compose([
    transforms.Resize(64),  # Resize to 224x224
    transforms.CenterCrop(64),  # Center crop to maintain the aspect ratio
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Apply data augmentation and normalization to datasets
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=transform_train)
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=transform_test)


In [19]:
# account for unlabeled data ()

# https://cdn-uploads.piazza.com/paste/jcwopj0bkHVK/4e633d4b629ae6f0c6630bba588e0d783805da7fd03580939c93a350d54adde5/NNDL_Multi-label_Classification_Competition.pdf

In [20]:
# train_ann_df = pd.read_csv('train_data.csv')
# super_map_df = pd.read_csv('superclass_mapping.csv')
# sub_map_df = pd.read_csv('subclass_mapping.csv')

# train_img_dir = 'train_shuffle'
# test_img_dir = 'test_shuffle'

# image_preprocessing = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0), std=(1)),
# ])

# Create train and val split
# train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
# test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [42]:
# 
class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cpu'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        for i, data in enumerate(self.train_loader):
            # print('1')
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)
            # print('2')
            self.optimizer.zero_grad()
            # print('3')
            super_outputs, sub_outputs = self.model(inputs)
            # print('4')
            # prob dist = [0.2, 0.05, ...]

            # threasholding, 
            loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                super_outputs, sub_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                super_outputs, sub_outputs = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                test_predictions['image'].append(img_name[0])
                test_predictions['superclass_index'].append(super_predicted.item())
                test_predictions['subclass_index'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('example_test_predictions.csv', index=False)

        if return_predictions:
            return test_predictions



# 
class SubclassTrainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cpu'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        for i, data in enumerate(self.train_loader):
            # print('1')
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)
            # print('2')
            self.optimizer.zero_grad()
            # print('3')
            # super_outputs, sub_outputs = self.model(inputs)
            sub_outputs = self.model(inputs)
            # print('4')
            # prob dist = [0.2, 0.05, ...]

            # threasholding, 
            loss = self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                sub_outputs = self.model(inputs)
                loss = self.criterion(sub_outputs, sub_labels)
                # _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                # super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        # print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=True, return_predictions=True):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                sub_outputs = self.model(inputs)
                # _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                test_predictions['ID'].append(img_name[0])
                # test_predictions['superclass_index'].append(super_predicted.item())
                test_predictions['Target'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('example_test_predictions.csv', index=False)

        if return_predictions:
            return test_predictions

In [36]:
# pretrained model

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18  # Using a smaller ResNet variant for 32x32 images
from torchvision.datasets import ImageFolder

# Load the pretrained ResNet model
model = resnet18(pretrained=True)

# Modify the last layers for your specific task
num_classes = 88  # Adjust based on your task
in_features = model.fc.in_features

# Replace the fully connected layer with your custom layers
model.fc = nn.Sequential(
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)


#CASE 1
class CustomResNet(nn.Module):
    def __init__(self, num_superclasses, num_subclasses):
        super(CustomResNet, self).__init__()
        self.resnet = resnet18(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_superclasses + num_subclasses)
        )
        self.num_superclasses = num_superclasses
        self.num_subclasses = num_subclasses

    def forward(self, x):
        x = self.resnet(x)
        super_outputs, sub_outputs = x[:, :self.num_superclasses], x[:, self.num_superclasses:]
        return super_outputs, sub_outputs

num_superclasses = 4
num_subclasses = 88

model = CustomResNet(num_superclasses, num_subclasses)


# CASE 2


import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

# Define the model
# class CustomResNet(nn.Module):
#     def __init__(self, num_classes):
#         super(CustomResNet, self).__init__()
#         self.resnet = resnet50(pretrained=True)
#         in_features = self.resnet.fc.in_features
#         self.resnet.fc = nn.Identity()  # Remove the last fully connected layer

#         # Add custom layers
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(in_features, 512)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(512, num_classes)

#     def forward(self, x):
#         x = self.resnet(x)
#         x = self.flatten(x)
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

# Instantiate the model
# model = CustomResNet(num_classes=88)


In [39]:
# Init model and trainer
device = 'cpu'
model = ResNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [40]:
# Training loop
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    # torch.cuda.empty_cache()
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1


ValueError: too many values to unpack (expected 2)

In [28]:
# predictions = trainer.test(save_to_csv=True, return_predictions=True)

print(predictions)

prediction_subclass = predictions

# Rename columns
# prediction_subclass.columns = ["ID", "Target"] + list(df.columns[2:])



# '''
# This simple baseline scores the following test accuracy

# Superclass Accuracy
# Overall: 43.83 % # 30%
# Seen: 61.11 %
# Unseen: 0.00 %

# Subclass Accuracy
# Overall: 2.03 % # above 7 target
# Seen: 9.56 %
# Unseen: 0.00 %
# '''

           image  superclass_index  subclass_index
0          0.jpg                 0              27
1          1.jpg                 0              72
2          2.jpg                 0              14
3          3.jpg                 0              27
4          4.jpg                 2              57
...          ...               ...             ...
12372  12372.jpg                 1              64
12373  12373.jpg                 0              77
12374  12374.jpg                 1              80
12375  12375.jpg                 1              77
12376  12376.jpg                 1              65

[12377 rows x 3 columns]
